## 使用 `czsc` 进行选股
---


In [1]:
!export http_proxy=http://nas.vvmm.ink:7890,https_proxy=http://nas.vvmm.ink:7890,all_proxy=socks5://nas.vvmm.ink:7891;
!pip install ta-lib mplfinance -i https://pypi.mirrors.ustc.edu.cn/simple
!pip install tqdm -i https://pypi.mirrors.ustc.edu.cn/simple
!pip install numba -i https://pypi.mirrors.ustc.edu.cn/simple
!pip install git+git://github.com/sohunjug/czsc.git

     |█████████████████████████████▍  | 245 kB 4.4 kB/s eta 0:00:05ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/pip/_vendor/urllib3/response.py", line 425, in _error_catcher
    yield
  File "/usr/local/lib/python3.8/site-packages/pip/_vendor/urllib3/response.py", line 507, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.8/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "/usr/local/lib/python3.8/http/client.py", line 454, in read
    n = self.readinto(b)
  File "/usr/local/lib/python3.8/http/client.py", line 498, in readinto
    n = self.fp.readinto(b)
  File "/usr/local/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/usr/local/lib/python3.8/ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/local/lib/python3.8/ssl.py", line 1099, in read
    return

In [1]:
import sys
sys.path.insert(0, ".")
sys.path.insert(0, "..")

import czsc
print(czsc.__version__)
from czsc.data.bs import *
from datetime import datetime
from typing import List
import traceback
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from czsc.analyze import KlineAnalyze
import tushare as ts
# 如果是第一次使用需要设置 token
ts.set_token("565ee6d69fb85cb0bc7fdfc8dea4b8ce1f18366f30e8d23a253bb9cf")

0.5.9
login success!
login respond error_code:0
login respond  error_msg:success


In [ ]:
# 导入 Tushare 数据（推荐使用）
from czsc.data.ts import *


In [2]:
# 导入聚宽数据
from czsc.data.jq import *

# 如果是第一次使用需要设置 token
set_token("18688989717", "hwqdir5.")

### 缠论日线笔中枢三买选股
---

条件描述： 最近五笔走势，前三笔构成中枢，第四笔离开中枢，第五笔向下不回中枢



In [3]:
def selector1(symbols: List, freq="d"):
    """输入股票列表，输入符合买点定义的股票"""
    res = []
    for symbol in tqdm(symbols, desc="缠论日线笔中枢三买选股"):
        try:
            kline = get_kline(symbol=symbol, end_date=datetime.now(), freq=freq, count=500)
            ka = KlineAnalyze(kline, ma_params=(5, 34, 120, 233), bi_mode="new")
            points = ka.bi_list[-6:]
            
            if len(points) == 6 and points[-1]['fx_mark'] == "d":
                zs_g = min([x['bi'] for x in points[:4] if x['fx_mark'] == 'g'])
                zs_d = max([x['bi'] for x in points[:4] if x['fx_mark'] == 'd'])
                
                if points[-1]['bi'] > zs_g > zs_d:
                    res.append(symbol)
                    
        except:
            print("{} 分析失败".format(symbol))
            traceback.print_exc()
    return res

### 日线笔中枢选股
---

中枢如果能当下确认，基本逻辑如下：
1. ma 233<close<ma 60 ---得 codes_list1
2. codes_list1中选择 符合日线笔中枢形态的，得codes_list2
    * 2.1 笔中枢形态：方向下: 日线中枢完成，第四笔底分型
3. codes_list2 中中枢第一笔到第四笔得日期跨度 date1
4. date1时间跨度中，codes_list2 ma233 cross的个数，并标注 codes_list3

In [4]:
def cross_number(x1, x2):
    """输入两个序列，计算 x1 下穿 x2 的次数"""
    x = np.array(x1) < np.array(x2)
    num = 0
    for i in range(len(x)-1):
        b1, b2 = x[i], x[i+1]
        if b2 and b1 != b2:
            num += 1
    return num

x1 = [1, 1, 3, 4, 5, 12, 9, 8]
x2 = [2, 2, 1, 5, 8, 9, 10, 3]
print(cross_number(x1, x2))

2


In [5]:
def selector(symbols: List, freq="d"):
    """输入股票列表，输入符合买点定义的股票"""
    res = []
    for symbol in tqdm(symbols, desc="缠论选股"):
        try:
            kline = get_kline(symbol=symbol, end_date=datetime.now(), freq=freq, count=500)
            ka = KlineAnalyze(kline, ma_params=(5, 34, 60, 250), bi_mode="new")

            if ka.ma[-1]['ma60'] >= ka.latest_price >= ka.ma[-1]['ma250']:
                # print("{} 满足条件1：ma60 > close > ma233".format(symbol))
                points = ka.bi_list[-7:]
                
                if len(points) == 7 and points[-1]['fx_mark'] == 'd':
                    zs_g = min([x['bi'] for x in points[2:6] if x['fx_mark'] == 'g'])
                    zs_d = max([x['bi'] for x in points[2:6] if x['fx_mark'] == 'd'])
                    
                    if zs_g > zs_d:
                        # print("{} 满足条件2：向下中枢完成".format(symbol))
                        date_span = [points[-5]['dt'], points[-1]['dt']]
                        low = [x['low'] for x in ka.kline_raw if date_span[1] >= x['dt'] >= date_span[0]]
                        ma_ = [x['ma250'] for x in ka.ma if date_span[1] >= x['dt'] >= date_span[0]]
                        num = cross_number(low, ma_)
                        res.append({"symbol": symbol, "cross_num": num})
        except:
            print("{} 分析失败".format(symbol))
            traceback.print_exc()
    return res

## 执行选股
---

In [5]:
# 使用 tushare 数据在创业板综指上选股
symbols = get_index_stocks("hs300")
#print(symbols)
selected = selector1(symbols)

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)

../czsc/data/bs.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k[col] = k[col].apply(round, args=(2,))



选股结果： ['sh.600111', 'sh.600406', 'sh.600009', 'sh.601117', 'sz.000651', 'sh.600362', 'sz.002008', 'sh.601111', 'sh.601919', 'sh.600809', 'sh.600893', 'sz.000768', 'sh.600309', 'sz.002415', 'sh.601021', 'sh.600690', 'sh.601398', 'sh.601318', 'sh.600989', 'sh.603833', 'sz.002600', 'sh.600703', 'sh.601600', 'sh.601899', 'sz.000786', 'sh.600176', 'sh.600025', 'sh.603993', 'sh.600660', 'sh.601601', 'sz.002601', 'sz.000333', 'sz.002460']


In [7]:
pd.DataFrame(selected)

,0
0,sh.600111
1,sh.600406
2,sh.600009
3,sh.601117
4,sz.000651
5,sh.600362
6,sz.002008
7,sh.601111
8,sh.601919
9,sh.600809


In [ ]:
# 使用聚宽数据在创业板综指上选股
#symbols = get_index_stocks("399006.XSHE")
symbols = get_index_stocks("000300.XSHG")
#print(symbols)
selected = selector(symbols)

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)

In [6]:
# 使用 tushare 数据在创业板综指上选股
symbols = get_index_stocks("000300.XSHG")
#print(symbols)
selected = selector1(symbols, "D")

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)


选股结果： ['000157.XSHE', '000333.XSHE', '000651.XSHE', '000768.XSHE', '000786.XSHE', '002008.XSHE', '002415.XSHE', '002460.XSHE', '002601.XSHE', '600009.XSHG', '600019.XSHG', '600025.XSHG', '600111.XSHG', '600176.XSHG', '600362.XSHG', '600406.XSHG', '600660.XSHG', '600690.XSHG', '600703.XSHG', '600809.XSHG', '600893.XSHG', '600989.XSHG', '601021.XSHG', '601111.XSHG', '601117.XSHG', '601318.XSHG', '601398.XSHG', '601600.XSHG', '601601.XSHG', '603833.XSHG', '688036.XSHG']


In [9]:
# 使用 tushare 数据在创业板综指上选股
symbols = get_index_stocks("000300.XSHG")
#print(symbols)
selected = selector1(symbols, "15min")

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)


选股结果： ['000625.XSHE', '000776.XSHE', '002008.XSHE', '002032.XSHE', '002129.XSHE', '002230.XSHE', '002410.XSHE', '300003.XSHE', '300059.XSHE', '300408.XSHE', '300413.XSHE', '300676.XSHE', '600018.XSHG', '600019.XSHG', '600176.XSHG', '600196.XSHG', '600299.XSHG', '600369.XSHG', '600438.XSHG', '600489.XSHG', '600887.XSHG', '600893.XSHG', '601088.XSHG', '601555.XSHG', '601618.XSHG', '601985.XSHG', '603087.XSHG', '603288.XSHG']


In [ ]:
# 使用 tushare 数据在创业板综指上选股
symbols = get_index_stocks("000300.XSHG")
#print(symbols)
selected = selector1(symbols, "30min")

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)

In [11]:
# 使用 tushare 数据在创业板综指上选股
symbols = get_index_stocks("000300.XSHG")
#print(symbols)
selected = selector1(symbols, "60min")

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)


选股结果： ['002508.XSHE', '002607.XSHE', '600519.XSHG', '600600.XSHG', '601577.XSHG', '603392.XSHG', '603899.XSHG']


In [12]:
# 使用 tushare 数据在创业板综指上选股
symbols = get_index_stocks("000300.XSHG")
#print(symbols)
selected = selector1(symbols, "5min")

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)


选股结果： ['000100.XSHE', '000725.XSHE', '002230.XSHE', '002271.XSHE', '002304.XSHE', '002352.XSHE', '002555.XSHE', '002600.XSHE', '002812.XSHE', '002821.XSHE', '300433.XSHE', '600031.XSHG', '600332.XSHG', '603087.XSHG', '603156.XSHG', '603259.XSHG', '603833.XSHG']
